[CMake]: https://cmake.org/
# [CMake][]によるBuild環境構築

directory 構成を以下のようにします。  
"*.[ch]"などのソースコードは **src** ディレクトリ以下に配置し、  
"*.{cpp,hpp}"などのテストコードは **test** ディレクトリ以下に配置するようにします。

```Console:toplevel
.
├── CMakeLists.txt
├── config
├── doc
├── src
└── test
```


[CMake]: https://cmake.org/
# 基本的な設定項目
top-directoryにある`CMakeLists.txt`では、
まず、以下の基本的なものを設定します。

```CMake:<top-directory>/CMakeLists.txt
cmake_minimum_required(VERSION 3.11)
project(test_library)
```

* `cmake_minimum_required()`で [CMake][]の要求バージョンを指定します。
手元で使っているバージョンが`3.11`だからですが、 特別な機能を使っているわけではないです。 
そのため、異なるバージョンでも大丈夫だと思います。
* `project()`でプロジェクトの名前を指定します。
ここで指定した名前は、`CMAKE_PROJECT_NAME`という変数に設定され、使用できます。


# CMAKE変数とCACHE変数
`CACHE` 指定がない場合は、そのファイル内でしか使用できませんが、
`CACHE` 指定すると、指定した以降の他の`CMakeLists.txt`,sub-directoryからでも使用できます。

```CMake:<top-directory>/CMakeLists.txt
set(proj_topdir ${PROJECT_SOURCE_DIR})
set(src_dir ${proj_topdir}/src CACHE FILEPATH "source directory")
```

* 変数`proj_topdir`に、CMAKE変数`PROJECT_SOURCE_DIR`の値を設定しています。
CMAKE変数`${PROJECT_SOURCE_DIR}`には、現在のプロジェクトのトップディレクトリが設定されています。
* 変数`src_dir`に本プロジェクトのsource-directoryの位置を設定しています。
source-directory の sub-directory から直下のヘッダファイルを`include`する必要があるため、
後で使用できるように`CACHE`で指定します。


[jansson]: http://www.digip.org/jansson/
[GoogleTest]: https://github.com/google/googletest
# 外部プロジェクトの取り込み

`CMake`は外部プロジェクトを取り込むことができます。

本プロジェクトでは設定ファイルとしてjson形式のファイルを用いており、
C言語のjsonライブラリである[jansson][]を利用して、設定ファイルの読み込みを行います。
また、UnitTestとして[GoogleTest][]を使用します。

```CMake:<top-directory>/CMakeLists.txt
find_package(Threads REQUIRED)
find_package(GTest 1.8.1 REQUIRED)
find_package(jansson REQUIRED)
```


[jansson]: http://www.digip.org/jansson/
[GoogleTest]: https://github.com/google/googletest
[CMake]: https://cmake.org/
# 外部プロジェクトの取り込み (cont'd)

`find_package()`コマンドにより使用できる外部プロジェクトは、
[CMake][]によるパッケージ探索に対応している必要があります。

[jansson][],[GoogleTest][]ともに、[CMake][]に対応しています。  
[jansson][]-2.12, [GoogleTest][]-1.8.1 のソースコードをダウンロードして、
インストールすると、`/usr/local/lib/cmake`以下に `jansson`, `GTest` という
名前のディレクトリが作成されており、`*.cmake` という名前のファイルが置かれています。  
これが`find_package()`の探索対象となり、インクルードパス・リンクパスが設定されます。


[GoogleTest]: https://github.com/google/googletest
# [GoogleTest][]

本プロジェクトでは、ユニットテストに[GoogleTest][]を使用します。

[GoogleTest][]は、C/C++用のテストフレームワークです。
[GoogleTest][]の`gtest`,`gtest_main`,`gmock`を使用します

* `gtest`はTest用のフレームワークで、
* `gtest_main`は毎回テスト用の`main()`関数を書かなくても良いようにしてくれるものです。
* `gmock`はTest用のMockフレームワークです。


# sub-directory の追加

top-directory から "*.[ch]" などの source-code を `src` directory 以下に、  
"*.{cpp,hpp}"などの test-code を `test` directory 以下に、それぞれ配置することにします。

また、`enable_testing()`で`CTest`を有効化します。
`CTest`はテストの実行と結果表示をしてくれるコマンドです。

**注意事項**:`enable_testing()`はtop-directoryの`CMakeLists.txt`に書く必要があります。

```CMake:<top-directory>/CMakeLists.txt
add_subdirectory(src)
if (NOT without-test)
    enable_testing()
    add_subdirectory(test)
endif()
```

# include directory の設定

source-code で include するヘッダファイルの置き場所を
`include_directories()` を使って指定します。

```CMake:<top-directory>/src/CMakeLists.txt
include_directories(${src_dir})
```

先に CACHE を使って指定した変数は `${}` で参照することができます。　  
または、明示的に `$CACHE{}` を使って参照することもできます。


# link directory の設定

object-file が link する他のオブジェクトファイルの置き場所を
`link_directories()` を使って指定します。

```CMake:<top-directory>/src/CMakeLists.txt
link_directories(${lib_dir})
```


[CMake]: https://cmake.org/
# [CMake][]の変数表示

CMake を使っていて、意図通りに動作しないことがあります。  
そのようなときには、[CMake][]で使用されている変数を全て出力することで、  
意図通りに動作しない原因を見つける手助けとなります。

```CMake:<top-directory>/src/CMakeLists.txt
get_cmake_property(_variable_names VARIABLES)
```
まず、`get_cmake_property()`で `VARIABLES` プロパティに含まれている変数群を
`_variable_names` に格納します。


[CMake]: https://cmake.org/
# [CMake][]の変数表示 (cont'd)

取得した変数群を`foeeach()`文で繰り返し、`message()`を使って、
変数名`${_variable_name}`と
変数名の値`${${_variable_name}}`を表示しています。
```CMake:<top-directory>/src/CMakeLists.txt
foreach(_variable_name ${_variable_names})
    message(STATUS "${_variable_name}=${${_variable_name}}")
endforeach()
```

##### References
* [CMakeで設定されているすべての変数を出力する](http://zashikiro.hateblo.jp/entry/2014/05/17/001314)


# Comment out

一行コメントアウトは`#`です。  
複数行にわたるコメントアウトは、`#[[`から開始して`]]`で閉じます。

```CMake:<top-directory>/src/CMakeLists.txt
#[[
message(STATUS "*** src: dump start cmake variables ***")
get_cmake_property(_variable_names VARIABLES)
foreach(_variable_name ${_variable_names})
    message(STATUS "${_variable_name}=${${_variable_name}}")
endforeach()
message(STATUS "*** src: dump end ***")
]]
```


# compile 時の define 追加

コンパイラに define を渡したいときは、`add_definitions()`を使って渡すことができます。

```CMake:<top-directory>/src/CMakeLists.txt
add_definitions("-DCSOMEIP_CONFIG_PATH=\"${CSOMEIP_CONFIG_PATH}\"")
```


# compile option の追加

コンパイラにオプションを渡したいときは、`CMAKE_C_FLAGS`にコンパイラオプションを設定すれば、
コンパイル時にコンパイラにオプションが渡されます。

```CMake:<top-directory>/src/CMakeLists.txt
set(cmake_c_warning_flags "-Wall -W -Wno-sign-compare")
set(cmake_c_warning_flags "${cmake_c_warning_flags} -Wno-unused-parameter")
set(CMAKE_C_FLAGS "-std=c99 -std=c11 -std=gnu99 -g3 -O0 \
                   ${cmake_c_warning_flags})
```


# compile 設定

以下では、コンパイル対象のファイルを`file()`で`HOGE_SOURCES`変数に設定しています。
`GLOB`コマンドを使うと、正規表現に似ているけどもっと簡単化した式が使えます。
ここでは明示的にファイルを指定しているため、正規表現を使っていませんが、
`*, ?, [a-Z]`などの特殊文字を使うことが出来ます。

```CMake:<top-directory>/src/CMakeLists.txt
file(GLOB HOGE_SOURCES hoge.c)
```


# compile 設定 (cont'd)

`add_library()`でライブラリを作成します。
ここでは、静的ライブラリ(`STATIC`)`hoge`を`${HOGE_SOURCES}`変数に格納した
ファイル群から作成します。
`STATIC`の他に動的ライブラリ(`SHARED`)やプラグイン(`MODULE`:dlopen()などでロード)が使えます。

```CMake:<top-directory>/src/CMakeLists.txt
add_library(libhoge-static STATIC ${HOGE_SOURCES})
```


# test compile 設定

ここでは、テストコード`Test_hoge.cpp`をコンパイルし、  
`add_executable()`を使って、`test_hoge` という実行ファイルを生成しています。

そして、その実行ファイルにリンクするライブラリ群を
`target_link_libraries()`を使ってリンクさせています。
リンクするライブラリ順序は、依存されるライブラリを後に書きます。

以下の例では、`test_hoge`は`jansson`に依存しています。
(`gtest gmock gtest_main pthread`は
自前のライブラリには依存していないので、どこに置いても構いません。)

```CMake:<top-directory>/test/CMakeLists.txt
file(GLOB HOGE_SOURCES Test_hoge.cpp)
add_executable(test_hoge ${HOGE_SOURCES})
target_link_libraries(test_hoge jansson
                      gtest gmock gtest_main pthread)
```


[GoogleTest]: https://github.com/google/googletest
# test compile 設定 (cont'd)

本プロジェクトでは、`CTest`を使います。`CTest`は、テストの実行を支援するテストランナーです。
`CTest`に実行するテストコマンドを登録しておくと、`CTest`によって、それら登録されたテストが実行されます。

テストコマンドの登録には`add_test()`を使います。
また、テストコマンドにラベルを設定(`set_tests_properties()`)しておくと、
`CTest -L <LabelName>`のように、ラベル名で実行するテストを指定することができます。

```CMake:<top-directory>/test/CMakeLists.txt
add_test(NAME Test_hoge COMMAND test_hoge)
set_tests_properties(Test_hoge PROPERTIES LABELS Test_hoge)
```


# install 指定

`TARGETS`フォームでは、プロジェクトの成果物をインストールします。
インストールする先のディレクトリは、`DESTINATION`で指定します。

`DESTINATION`は、full-pathで指定するか、relative-pathの場合は、
`CMAKE_INSTALL_PREFIX`で指定したディレクトリが使用されます。

以下の例では、`${CMAKE_INSTALL_PREFIX}/lib`にインストールされます。

```CMake:<top-directory>/src/CMakeLists.txt
install(TARGETS libhoge DESTINATION lib)
install(TARGETS libhoge-static DESTINATION lib)
```


# install 指定 (cont'd)

`FILES`フォームでは、プロジェクトで用いるファイルをインストールします。
以下では使用していませんが、`FILES`フォームでは、ファイルのpermissionsなどの指定もできます。

```CMake:<top-directory>/src/CMakeLists.txt
install(FILES ${API_HEADERS} DESTINATION include)
```


[gcovr]: https://www.gcovr.com/en/stable/
# Coverage 算出

ソフトウェア開発においては、開発メトリクスの集計や品質を計測するために
テストカバレッジを計測することがあります。

コンパイル時にカバレッジ用のオプションを追加することで、
後で[gcovr][]を使って、テストカバレッジを算出することができます。

```CMake:<top-directory>/src/CMakeLists.txt
set(test_coverage_flags "-fprofile-arcs -ftest-coverage")
set(CMAKE_C_FLAGS ${cmake_c_warning_flags} ${test_coverage_flags}")
set(CMAKE_EXE_LINKER_FLAGS ${test_coverage_flags})
```
